In [40]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import collections
import math
import os
import random
import zipfile
import pyodbc
import sys
import re
import preprocessor

[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

['Microsoft Access Driver (*.mdb, *.accdb)']

In [41]:
def preprocess(paragraph):
    #sentences = tokenizer.tokenize(paragraph)
    sentences = string.split(paragraph, '\n') 
   
    return sentences

In [42]:
def read_words():
    words = []
    conn_str = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\PhD\Webtrace\WebTrace\WebTrace\WebTrace\PCIWebTrace\PCIWebTrace.mdb;')
    cnxn = pyodbc.connect(conn_str)
    #cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf8')
    #cnxn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf8')
   
    crsr = cnxn.cursor()
    sql = "SELECT PullRequestDescription from PullRequest"
    crsr.execute(sql)
    # show the result
    result = crsr.fetchall();
    for item in result:      
        words.append(preprocessor.preprocessNoCamel(str(item)))
    return words

In [43]:
vocabulary = read_words()
len(vocabulary)

402